# Initialization

Test notebook for the C-MAPPS benchmark. Approach using MLP. 

First we import the necessary packages and create the global variables.

In [106]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.font_manager
import matplotlib
import seaborn as sns
import pandas as pd
import logging
import random
#import plottingTools
#from datetime import datetime
#from sklearn.covariance import EllipticEnvelope
#from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn.dummy import DummyClassifier
#from sklearn.model_selection import train_test_split, cross_validate
#from sklearn.neural_network import MLPClassifier
#from mpl_toolkits.mplot3d import Axes3D
#from dataManagement import DataManagerDamadics
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

from IPython.display import display, HTML
%matplotlib notebook

global constRUL

constRUL = 125
time_window = 30
rul_vector = None

In [4]:
def compute_training_RUL(df_row, *args):
    
    global constRUL
    rul_vector = args[0]
    
    if rul_vector[int(df_row['Unit Number']) - 1] - df_row['Cycle'] > constRUL:
        return constRUL
    else:
        return rul_vector[int(df_row['Unit Number']) - 1] - df_row['Cycle']

In [5]:
def get_X_y_from_df(df, time_window, features, num_units, dataset_type):
    
    n_m = df.shape[0]
    n_x = len(features)
    
    df_values = df[features].values
    targets = df['RUL'].values
    n_m = 0
    n_X = len(features)
    df_unit_values = []
    targets_unit = []
    num_samples_unit = []
    
    #Count number of elements at each group so that we can create the matrix to hold them all. 
    #Also store each matrix in temporary arrays to access them faster
    for i in range(1,num_units+1):
        
        df_unit = df.loc[df['Unit Number'] == i]
        df_unit_values.append(df_unit[features].values) #is this a view or a copy of the df?
        targets_unit.append(df_unit['RUL'].values) #is this a view or a copy of the df?
        num_samples_unit.append(df_unit.shape[0])
        n_m = n_m + num_samples_unit[i-1]-time_window+1
    
    #Create the numpy arrays to hold the features
    if (dataset_type == 'train' or dataset_type == 'cross_validation'):
        X, y = np.empty([n_m, n_x*time_window]), np.empty([n_m, 1])
    else:
        X, y = np.empty([num_units, n_x*time_window]), np.empty([num_units, 1])
        
    k = 0
    
    #Create the feature matrix by moving the time window for each type of engine.
    for i in range(num_units):
    
        if (dataset_type == 'train' or dataset_type == 'cross_validation'):
            for j in range(num_samples_unit[i]-time_window+1):

                time_window_samples = df_unit_values[i][j:j+time_window]
                X[k,:] = np.squeeze(time_window_samples.reshape(1,-1))
                y[k] = targets_unit[i][j+time_window-1]
                k = k + 1
        else:
            #print(dataset_type)
            time_window_samples = df_unit_values[i][-time_window:]
            X[k,:] = np.squeeze(time_window_samples.reshape(1,-1))
            k = k + 1
    
    return X, y

# Retrieve and Reshape data

Get the data from the text files, store it in a Pandas Dataframe and reshape it as appropiately.

In [55]:
def retrieve_and_reshape_data(from_file, columns, time_window, dataset_type):
    '''
    5    T2        - Total temperature at fan inlet      R
    6    T24       - Total temperature at lpc outlet     R
    7    T30       - Total temperature at hpc outlet     R
    8    T50       - Total temperature at LPT outlet     R
    9    P2        - Pressure at fan inlet               psia
    10   P15       - Total pressure in bypass-duct       psia
    11   P30       - Total pressure at HPC outlet        psia
    12   Nf        - Physical fan speed                  rpm
    13   Nc        - Physical core speed                 rpm
    14   epr       - Engine Pressure ratio (P50/P2)      --
    15   Ps30      - Static Pressure at HPC outlet       psia
    16   phi       - Ratio fuel flow to Ps30             pps/psi
    17   NRf       - corrected fan speed                 rpm
    18   NRc       - Corrected core speed                rpm
    19   BPR       - Bypass ratio                        --
    20   farB      - Burner fuel-air ratio               --
    21   htBleed   - Bleed enthalpy                      --
    22   Nf_dmd    - Demanded fan speed                  rpm
    23   PCNfR_dmd - Demanded corrected fan speed        rpm
    24   W31       - HPT coolant bleed                   lbm/s
    25   W32       - LPT coolant bleed                   lbm/s
    '''

    
    df = pd.read_csv(from_file ,sep='\s+',header=None)

    col_names = {0:'Unit Number', 1:'Cycle', 2:'Op. Settings 1', 3:'Op. Settings 2', 4:'Op. Settings 3', 5:'T2',
                6:'T24', 7:'T30', 8:'T50', 9:'P2', 10:'P15', 11:'P30', 12:'Nf', 13:'Nc', 14:'epr', 15:'Ps30', 
                16:'phi', 17:'NRf', 18:'NRc', 19:'BPR', 20:'farB', 21:'htBleed', 22:'Nf_dmd', 23:'PCNfR_dmd', 
                24:'W31', 25:'W32'}

    df.rename(columns=col_names, inplace=True)

    gruoped_by_unit = df.groupby('Unit Number')
    rul_vector = gruoped_by_unit.size().values
    num_units = len(gruoped_by_unit)

    df['RUL'] = df.apply(compute_training_RUL, axis = 1, args=(rul_vector,))
    selected_features_rul = columns[:]
    selected_features_rul.extend(['Unit Number', 'RUL'])
    df_selected_features = df[selected_features_rul]

    X, y = get_X_y_from_df(df_selected_features, time_window, selected_features, num_units, dataset_type)
    
    #display(df_selected_features.head(5))
    
    return X, y

    '''display(df_selected_features)
    print(X.shape)
    print(X)
    print(y.shape)
    print(y)'''

# Keras model

We will use a very simple ANN for this example. The model is Dense(ReLU, 100)->Dense(ReLu, 100)->Dense(Linear, 1)

In [85]:
def RULmodel(input_shape):
    
    print(input_shape)
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(1000, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(500, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.5))
    #model.add(Dense(100, activation='relu', name='fc3'))
    #model.add(Dropout(0.5))
    #model.add(Dense(10, activation='relu', name='fc4'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear', name='out'))
    
    #create a placeholder for the input
    #X_input = Input(shape=(input_shape))
    
    #Create the layers
    #X = Dense(100, activation='relu', name='fc1')(X_input)
    #X = Dense(100, activation='relu', name='fc2')(X)
    #X = Dense(1, activation='linear', name='out')(X)
    
    # Create model. This creates the Keras model instance, you'll use this instance to train/test the model.
    #model = Sequential(inputs = X_input, outputs = X, name='RUL')
    
    return model
    
    

# Retrieve the train and test data from the files

In [80]:
data_file_train = '../CMAPSSData/train_FD001.txt'
data_file_test = '../CMAPSSData/test_FD001.txt'

#min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
standardScaler = StandardScaler()
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

#Get the X and y matrices with the specified time window
X_train, y_train = retrieve_and_reshape_data(data_file_train, selected_features, time_window, 'train')
X_test, _ = retrieve_and_reshape_data(data_file_test, selected_features, time_window, 'test')
y_test = np.loadtxt("../CMAPSSData/RUL_FD001.txt")
y_test = np.array([x if x < constRUL else constRUL for x in y_test])
y_test = np.reshape(y_test, (y_test.shape[0], 1))

#Standardize the data
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)



In [81]:
print("Training data")
print(X_train.shape)
print(y_train.shape)
print("Testing data")
print(X_test.shape)
print(y_test.shape)
print("Training data")
print(X_train[-5:,:])
print(y_train[-5:,:])
print("Testing data")
print(X_test[-5:,:])
print(y_test[-5:,:])

Training data
(17731, 420)
(17731, 1)
Testing data
(100, 420)
(100, 1)
Training data
[[ 0.62199313  0.32071611  0.36067504 ...  0.45454545 -0.4488189
  -0.77474791]
 [ 0.20962199  0.14475703  0.60204082 ...  0.09090909 -0.7480315
  -0.24669791]
 [ 0.12714777  0.14475703  0.71428571 ...  0.63636364 -0.52755906
  -0.88893623]
 [ 0.03092784  0.14475703  0.78296703 ...  0.09090909 -0.76377953
  -0.51768215]
 [ 0.26460481  0.15191816  0.50039246 ...  0.27272727 -0.63779528
  -0.55120011]]
[[4.]
 [3.]
 [2.]
 [1.]
 [0.]]
Testing data
[[-0.72682927 -0.95827124 -0.89279113 ... -0.71428571  0.45238095
   0.84135021]
 [ 0.13170732 -0.44336811 -0.39149723 ...  0.42857143 -0.38095238
   0.14388186]
 [-0.08292683 -0.01266766 -0.3323475  ...  0.14285714 -0.02380952
   0.42025316]
 [-0.49268293 -0.23919523 -0.92310536 ... -0.71428571  0.42857143
   0.41476793]
 [-0.32682927 -0.11698957  0.06617375 ...  0.71428571 -0.16666667
  -0.31940928]]
[[125.]
 [ 82.]
 [ 59.]
 [117.]
 [ 20.]]


# Fit the keras model

Fit the Keras model to the data and determine its performance.

#Create the model
modelRUL = RULmodel(X_train.shape[1])

#Compile the model.
modelRUL.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ["accuracy"])

#Train the model.
modelRUL.fit(x = X_train, y = y_train, epochs = 200, batch_size = 128)

#Evaluate the model
preds = modelRUL.evaluate(x = X_test, y = y_test)

print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

# Deep Net approach

The approach used in the original paper.

# Auxiliary functions

In [110]:
def get_score(y_true, y_pred):
    s=0
    for i in range(len(y_true)):
        d = y_pred[i] - y_true[i]
        if d < 0:
            s+=math.e**(-d/13)-1
        else:
            s+=math.e**(d/10)-1
    return s

def step_decay(epoch):
    lrat = 0
    if epoch<200:
        lrat = 0.001
    else:
        lrat = 0.0001
    return lrat 

# Constants

In [111]:
FeatureN = 14
nb_epoch = 250
batch_size = 512
FilterN = 10
FilterL = 10
rmse,sco,tm = [], [], []

ConstRUL = 125
TW = 30
Dataset = '1'

# Retrieve the train and test data from the files

In [112]:
############ training samples ##################################

setTrain = {'1':100, '2':260, '3':100, '4':248}
setTest = {'1':100, '2':259, '3':100, '4':248}
nTrain = setTrain[Dataset]
nTest = setTest[Dataset]

data = [] 
for line in open("../CMAPSSData/train_FD00"+Dataset+".txt"):
    data.append(line.split())
data=np.array(data)
data = np.cast['float64'](data)
data_copy = data
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
data = min_max_scaler.fit_transform(data)   #Why scale the data with all of the "missing" rows.
num=[]
for i in range(nTrain):
    tmp = data[np.where(data_copy[:,0]==i+1),:][0][:, np.array([6,7,8,11,12,13,15,16,17,18,19,21,24,25])]
    num.append(tmp)
num=np.array(num)

label=[]
for i in range(nTrain):
    label.append([])
    length = len(num[i])
    for j in range(length):
        label[i].append(ConstRUL if length-j-1>=ConstRUL else length-j-1)
label = np.array(label)

samples,targets,noofsample = [],[],[]
for i in range(nTrain):
    noofsample.append(len(num[i])-TW+1)
    for j in range(noofsample[-1]):
        samples.append(num[i][j:j+TW,:])
        targets.append(label[i][j+TW-1])
samples = np.array(samples)
targets = np.array(targets)

################## testing data ###########################
data = [] 
for line in open("../CMAPSSData/test_FD00"+Dataset+".txt"):
    data.append(line.split())
data=np.array(data)
data = np.cast['float64'](data)
data_copy = data
data = min_max_scaler.transform(data)  #Why scale the data with all of the "missing" rows.
numt=[]
for i in range(nTest):
    tmp = data[np.where(data_copy[:,0]==i+1),:][0][:, np.array([6,7,8,11,12,13,15,16,17,18,19,21,24,25])]
    numt.append(tmp)
numt=np.array(numt)

samplet, count_miss = [],[]
for i in range(nTest):
    if len(numt[i])>=TW:
        samplet.append(numt[i][-TW:,:])
    else:
        count_miss.append(i)
samplet = np.array(samplet)

labelt = [] 
for line in open("../CMAPSSData/RUL_FD00"+Dataset+".txt"):
    labelt.append(line.split())
labelt = np.cast['int32'](labelt)
labelnew = []
for i in range(nTest):
    if i not in count_miss:
        #labelnew.append(labelt[i][0])
        labelnew.append(labelt[i][0] if labelt[i][0]<=ConstRUL else ConstRUL)
labelt = labelnew
labelt=np.array(labelt)

'''seed = 2222
np.random.seed(seed)
np.random.shuffle(samples)
np.random.seed(seed)
np.random.shuffle(targets)
samplet = samplet[np.argsort(labelt)]
labelt = labelt[np.argsort(labelt)]'''

'seed = 2222\nnp.random.seed(seed)\nnp.random.shuffle(samples)\nnp.random.seed(seed)\nnp.random.shuffle(targets)\nsamplet = samplet[np.argsort(labelt)]\nlabelt = labelt[np.argsort(labelt)]'

In [74]:
print("Training data")
print(samples.shape)
print(targets.shape)
print("Testing data")
print(samplet.shape)
print(labelt.shape)
print("Training data")
print(samples[-5:,:])
print(targets[-5:])
print("Testing data")
print(samplet[-5:,:])
print(labelt[-5:])

Training data
(17731, 30, 14)
(17731,)
Testing data
(100, 30, 14)
(100,)
Training data
[[[ 0.42168675  0.12579028  0.17049291 ...  0.33333333 -0.27131783
   -0.07953604]
  [ 0.06024096 -0.02419882  0.37812289 ...  0.16666667 -0.27131783
    0.01712234]
  [-0.01204819 -0.02419882  0.47467927 ...  0.16666667 -0.11627907
   -0.11101906]
  ...
  [ 0.51204819  0.14453891  0.52464551 ...  0.         -0.62790698
   -0.34217067]
  [ 0.3253012   0.26444299  0.67623228 ...  0.         -1.
   -0.17674676]
  [ 0.37349398  0.17462394  0.5658339  ...  0.5        -0.45736434
   -0.78099972]]

 [[ 0.06024096 -0.02419882  0.37812289 ...  0.16666667 -0.27131783
    0.01712234]
  [-0.01204819 -0.02419882  0.47467927 ...  0.16666667 -0.11627907
   -0.11101906]
  [-0.09638554 -0.02419882  0.53376097 ...  0.         -0.24031008
   -0.2761668 ]
  ...
  [ 0.3253012   0.26444299  0.67623228 ...  0.         -1.
   -0.17674676]
  [ 0.37349398  0.17462394  0.5658339  ...  0.5        -0.45736434
   -0.78099972]
  

# Keras model

CNN model. The model is Dense(ReLU, 100)->Dense(ReLu, 100)->Dense(Linear, 1)

In [113]:
def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

In [114]:
opt = Adam(lr=0, beta_1=0.5)
#DCNN = Model([input_layer], [y])
DCNN = RULCNNModel(TW, FeatureN)
#DCNN.compile(loss=get_score,optimizer=opt)
DCNN.compile(loss='mean_squared_error',optimizer=opt)
lrate = LearningRateScheduler(step_decay)
history = DCNN.fit(samples, targets,nb_epoch=nb_epoch, batch_size=batch_size,verbose=1, 
                   validation_data=(samplet, labelt), callbacks=[lrate])
    
#, TensorBoard(log_dir='tmp\\tan_4c_4')
    
#history = DCNN.fit(samples, targets,nb_epoch=nb_epoch, batch_size=batch_size,verbose=1, 
#                 validation_data=(samplet, labelt), callbacks=[lrate])
#history = DCNN.fit(samples, targets,nb_epoch=nb_epoch, batch_size=batch_size,verbose=0, callbacks=[lrate])
    
score = DCNN.evaluate(samplet, labelt, batch_size=batch_size, verbose=1)
#print('Test score:', score)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C1", padding="same", activation="tanh", kernel_initializer="glorot_normal")`
  
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C2", padding="same", activation="tanh", kernel_initializer="glorot_normal")`
  import sys
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C3", padding="same", activation="tanh", kernel_initializer="glorot_normal")`
  
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C4", padding="same", activation="tanh", kernel

Train on 17731 samples, validate on 100 samples
Epoch 1/250
17731/17731 [==============================] - 10s 574us/step - loss: 6572.4042 - val_loss: 5025.3726
Epoch 2/250
17731/17731 [==============================] - 10s 570us/step - loss: 5693.9979 - val_loss: 4600.8364
Epoch 3/250
10752/17731 [=================>............] - ETA: 3s - loss: 5303.0278

KeyboardInterrupt: 